In [2]:
import numpy as np
from numpy import linalg as LA
import scipy.linalg as sc
import time
import ot
import pylab as plt
import matplotlib

ImportError: No module named ot

## Решение транспортной задачи линейного программирования методом Синхорна

In [ ]:
def B(u, v, K, n):
    return (np.eye(n) * np.exp(u)) @ K @ (np.eye(n) * np.exp(v))

In [3]:
def count_B(eps, r, c, K, n):
    k = 0
    u = np.zeros(n)
    v = np.zeros(n)

    while LA.norm(B(u,v, K, n) @ np.ones(n) - r, 1) + LA.norm(B(u,v, K,n).T @ np.ones(n) - c, 1) > eps:
        if k % 2 == 0:
            u += np.log(r) - np.log(B(u, v, K, n) @ np.ones(n))
        else:
            v += np.log(c) - np.log(B(u, v, K, n).T @ np.ones(n))
            
        k += 1

        
    return B(u, v, K, n)

SyntaxError: invalid syntax (<ipython-input-3-c5b85bb86c0a>, line 6)

In [4]:
def proj(r, c, B, n):
    x_i = np.array([ min(r[i]/(B @ np.ones(n))[i], 1)  for i in range(n) ] )
    y_i = np.array([ min(c[i]/(B.T @ np.ones(n))[i], 1)  for i in range(n) ] )
    X = np.diag(x_i)
    B_1 = X @ B
    Y = np.diag(y_i)
    B_2 = B_1 @ Y
    err_r = r - B_2 @ np.ones(n)
    err_c = c - B_2.T @ np.ones(n)
    B_3 = B_2 + (err_r @ err_c.T) / LA.norm(err_r, 1)
    print("err={}".format((err_r @ err_c.T) / LA.norm(err_r, 1)))
    return B_3

In [5]:
def sinkhorn_solve(eps, r, c, C, n):
    gamma = eps / (4 * np.log(n))
    C = C * (-1) / gamma
    K = np.exp(C)
    eps_1 = eps / (8 * LA.norm(C, np.inf))
    r_1 = (1 - eps_1 / 8) * (r + eps_1/(n * (8 - eps_1) * np.ones(n)))
    c_1 = (1 - eps_1 / 8) * (c + eps_1/(n * (8 - eps_1) * np.ones(n)))
    B = count_B(eps_1 / 2, r_1, c_1, K, n)
    X = proj(r, c, B, n)
    return X

In [20]:
eps = 0.255
r = np.array([0.2, 0.3, 0.5])
c = np.array([0.6, 0.1, 0.3])
# a = np.array([0.2, 0.3, 0.5])
# b = np.array([0.45, 0.15, 0.4])
n = 3
C = np.array([[1, 2, 6], [7, 2, 1], [9,3,2]])
start = time.time()
X = sinkhorn_solve(eps, r, c, C, n)
stop = time.time()
print(X)
sum = 0
for i in range(n):
    for j in range(n):
        sum += X[i][j] * C[i][j]
        
print(sum)

print ("Время :{}".format(stop - start))

gamma = eps/(4* np.log(n))
start = time.time()
X = ot.sinkhorn(r, c, C, gamma)
stop = time.time()
print(X)

print(ot.sinkhorn2(r, c, C, gamma))
print ("Время :{}".format(stop - start))

err=5.42343914615116e-14
[[2.00000000e-01 5.42343915e-14 5.42343915e-14]
 [2.99999961e-01 9.84071634e-09 2.95220405e-08]
 [9.99635934e-02 9.99999902e-02 2.99999970e-01]]
4.099672025458258
Время :0.03719162940979004
[[2.00000000e-01 8.15163926e-54 9.27024746e-91]
 [2.99999961e-01 9.83707564e-09 2.95112269e-08]
 [1.00000039e-01 9.99999903e-02 2.99999971e-01]]
[4.10000004]
Время :0.005617380142211914


In [1]:
eps = [0.406, 0.35, 0.3, 0.255, 0.223]
time_s1 = [0.018610,0.025537490, 0.0283660, 0.0371916294,0.044445514]
time_s2 = [0.0056133270, 0.010893821, 0.0096976757, 0.00561738014, 0.0063638687]
log_time_s1 = np.vectorize(lambda i: np.log(time_s1[i]))(np.arange(5))
log_time_s2 = np.vectorize(lambda i: np.log(time_s2[i]))(np.arange(5))
log_eps = np.vectorize(lambda i: np.log(1./eps[i]))(np.arange(5))

plt.figure(figsize=(15,10))
plt.plot(log_eps, log_time_s1, label='sinkhorn function')
plt.plot(log_eps, log_time_s2, label='library function')
plt.legend(fontsize=15)
plt.xlabel("log(1/eps)")
plt.ylabel("log time")
plt.grid()
plt.show()

NameError: name 'np' is not defined

## Сложность метода
 ### Согласно статье, которую вы  нагуглили на ноуте Камиллы Давлетшиной, сложность метода составила $O(\frac{n^2}{\epsilon^2})$

In [4]:
def vec(X):
    rturn X.flatten('F')

SyntaxError: invalid syntax (<ipython-input-4-b9384d899718>, line 2)

In [5]:
def matrix(x):
    n = int(np.sqrt(x.shape[0]))
    return x.reshape(n, n)

In [ ]:
def apdag(eps1, eps2, L_0):
    alpha, beta, lam, eta, dzeta = np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n), np.zeros(n)
    tau = np.zeros(n)
    i, k = np.zeros(n), np.zeros(n)
    dif = np.zeros(n)
    M[0] = L_0
    while (f(x[k+1] + phi(eta[k+1])) > eps1) and (norm(A @ x[k+1] - b) > eps2):
        i[k+1] = 0
        k += 1
         x[k+1] = tau[k] * X(lam[k+1]) + (1 - tau[k]) * x[k]
        grad_phi = b - A @ X(lam[k+1])
        dif[k+1] = eta[k+1] - lam[k+1]
        while phi(eta[k+1]) > phi(lam[k+1]) + sum(grad_phi[:]*dif[k+1][:]) + M[k] / 2. * norm(dif[k+1]):
           
            M[k] = 2**(i[k] - 1) * M[k]
            alpha[k+1] = M[k] * alpha[k+1]**2
            tau[k] = alpha[k+1] / beta[k+1]
            lam[k+1] = tau[k] * dzeta[k] + (1 - tau[k]) * eta[k]
            dzeta[k+1] = dzeta[k] - alpha[k+1] * grad_phi
            eta[k+1] = tau[k] * dzeta[k+1] + (1 - tau[k]) * eta[k]
            
    return x[k+1], eta[k+1]
            